<a href="https://colab.research.google.com/github/SezhiyanP4597/blank-app/blob/main/Together_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q together langchain langchain-community faiss-cpu sentence-transformers pymupdf python-docx pytesseract Pillow gradio
!apt-get install -y poppler-utils tesseract-ocr


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.4/441

In [ ]:
import os
os.environ["TOGETHER_API_KEY"] = "tgp_v1_sSe_9il5x2P6DBSLP3GGzFYsa7X0g2H3k-rvXKR_ufA"


In [ ]:
import fitz
import pytesseract
from PIL import Image
from docx import Document

def extract_text(file):
    name = file.name.lower()

    if name.endswith(".pdf"):
        doc = fitz.open(file.name)
        return "\n".join(page.get_text() for page in doc)

    elif name.endswith(".docx"):
        return "\n".join(p.text for p in Document(file.name).paragraphs)

    elif name.endswith(".txt"):
        with open(file.name, "r", encoding="utf-8") as f:
            return f.read()

    elif name.endswith(('.png', '.jpg', '.jpeg')):
        image = Image.open(file.name)
        return pytesseract.image_to_string(image)

    return "Unsupported file format."


In [ ]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.llms import Together

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

vectorstore = None

def ingest_text(text):
    global vectorstore
    chunks = splitter.split_text(text)
    vectorstore = FAISS.from_texts(chunks, embedding_model)

def retrieve_context(query):
    return vectorstore.similarity_search(query, k=2) if vectorstore else []

llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.7,
    max_tokens=512,
    together_api_key=os.environ["TOGETHER_API_KEY"]
)

rag_prompt_template = PromptTemplate.from_template("""
You are a helpful assistant. Use only the provided context to answer the user's question.

Context:
{context}

Question:
{question}

Answer:
""")

def answer_question(query, return_sources=False):
    docs = retrieve_context(query)
    context = "\n\n".join(doc.page_content for doc in docs)[:3500]

    prompt = rag_prompt_template.format(context=context, question=query)
    response = llm.invoke(prompt, stop=["User:"])

    if return_sources:
        return response, [f"Doc {i+1}" for i in range(len(docs))]
    return response

def summarize_documents(text):
    prompt = f"Summarize the following in 3 sentences:\n\n{text[:3000]}"
    return llm.invoke(prompt)


/tmp/ipython-input-4-3243123039.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-4-3243123039.py:20: LangChainDeprecationWarning: The class `Together` was deprecated in LangChain 0.0.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-together package and should be used instead. To use it run `pip install -U :class:`~langchain-together` and import as `from :class:`~langchain_together import Together``.
  llm = Together(


In [ ]:
import gradio as gr


chat_history = []

def handle_upload(files):
    texts = []
    file_names = []

    for file in files:
        file_names.append(file.name)
        text = extract_text(file)
        texts.append(text)

    full_text = "\n\n".join(texts)
    ingest_text(full_text)
    summary = summarize_documents(full_text)

    return f"✅ Ingested: {', '.join(file_names)}\n\n📝 Summary:\n{summary}"

def handle_chat(user_input):
    response, sources = answer_question(user_input, return_sources=True)
    chat_history.append((user_input, f"{response}\n\n📚 Sources: " + ", ".join(sources)))
    return "", chat_history

def reset_chat():
    chat_history.clear()
    return []

def export_chat():
    md = "# 💬 Chat Transcript\n\n"
    for i, (q, a) in enumerate(chat_history, 1):
        md += f"**Q{i}:** {q}\n\n**A{i}:** {a}\n\n---\n"
    with open("chat.md", "w", encoding="utf-8") as f:
        f.write(md)
    return "chat.md"

with gr.Blocks(theme=gr.themes.Soft()) as app:
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📂 Upload Documents")
            file_input = gr.File(file_types=[".pdf", ".docx", ".txt", ".jpg", ".png"], file_count="multiple")
            upload_btn = gr.Button("Ingest Files")
            file_status = gr.Textbox(label="Upload Summary")

            gr.Markdown("### ⚙️ Options")
            clear_btn = gr.Button("🧹 Clear Chat")
            #export_btn = gr.Button("⬇ Download Chat")

        with gr.Column(scale=2):
            gr.Markdown("### 💬 Ask Questions About Your Files")
            chatbot = gr.Chatbot(label="RAG Chatbot", render_markdown=True)
            user_input = gr.Textbox(placeholder="Ask something...")
            send_btn = gr.Button("Send")

    upload_btn.click(fn=handle_upload, inputs=file_input, outputs=file_status)
    send_btn.click(fn=handle_chat, inputs=user_input, outputs=[user_input, chatbot])
    clear_btn.click(fn=reset_chat, outputs=chatbot)
    #export_btn.click(fn=export_chat, outputs=gr.File())

app.launch(share=True)


/tmp/ipython-input-7-4158556130.py:52: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="RAG Chatbot", render_markdown=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5e6ae12e63e443d70e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
